# // Getting NFL Data
___
The first step in getting this project underway is going to be getting **massive** amounts of NFL data from the web. I will be working in this notebook to "show my work" and for others to learn how to if they're curious. Ultimately, I'll also turn it into a regular `.py` Python script that you can run if you're so inclined.

For that, we're going to start out using the `requests` and `BeautifulSoup` libraries as well as API calls to glean information from:
- [Reddit recommendations](https://www.reddit.com/r/fantasyfootball/comments/34mbth/datasets_for_fantasy_football/)
- **nflgame** [here](http://wseaton.com/pulling-data-with-nflgame.html) and [here](https://pypi.org/project/nflgame/)
- [fantasydata](https://fantasydata.com/)
- [Pro Football Reference](https://www.pro-football-reference.com/)

In [2]:
# Importing our necessary libraries
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from bs4 import BeautifulSoup
%matplotlib inline

## Scraping Pro Football Reference: